## Demonstrate **hklpy2**'s API

A working notebook as the package is being developed.
***Anything*** could change.

In [1]:
import datetime
import hklpy2
import math

print(f"{datetime.datetime.now()}")
print(f"{hklpy2.__version__=}")
print(f"{hklpy2.solvers()=}")

2024-05-23 18:02:11.038338
hklpy2.__version__='0.0.14.dev0+g96622ed.d20240523'
hklpy2.solvers()={'hkl_soleil': 'hklpy2.backends.hkl_soleil:HklSolver', 'no_op': 'hklpy2.backends.no_op:NoOpSolver', 'th_tth': 'hklpy2.backends.th_tth_q:ThTthSolver'}


In [2]:
from ophyd import Component as Cpt
from ophyd import Kind
from ophyd import PseudoSingle
from ophyd import SoftPositioner

NORMAL_HINTED = Kind.hinted | Kind.normal

class Fourc(hklpy2.DiffractometerBase):
    """Test case."""

    # pseudo-space axes, in order expected by hkl_soleil E4CV, engine="hkl"
    h = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    # real-space axes, in order expected by hkl_soleil E4CV
    # using different names
    theta = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    chi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    phi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    ttheta = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    # pseudo-space extra axes used in a couple modes
    h2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    # real-space extra axis used in a couple modes
    psi = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    # another Component, not used (yet)
    energy = Cpt(SoftPositioner, limits=(5, 35), init_pos=12.4, kind=NORMAL_HINTED)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.set_solver("hkl_soleil", "E4CV", engine="hkl")
        self.operator.auto_assign_axes()

fourc = Fourc("", name="fourc")
print(f"{fourc=}")

fourc=Fourc(prefix='', name='fourc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['h', 'h.readback', 'h.setpoint', 'k', 'k.readback', 'k.setpoint', 'l', 'l.readback', 'l.setpoint', 'theta', 'chi', 'phi', 'ttheta', 'h2', 'h2.readback', 'h2.setpoint', 'k2', 'k2.readback', 'k2.setpoint', 'l2', 'l2.readback', 'l2.setpoint', 'psi', 'energy'], configuration_attrs=['geometry', 'solver', 'wavelength', 'h', 'k', 'l', 'h2', 'k2', 'l2'], concurrent=True)


In [3]:
print(f"{fourc.operator.solver.geometries()=}")

fourc.operator.solver.geometries()=['E4CH', 'E4CV', 'E6C', 'ESRF ID01 PSIC', 'K4CV', 'K6C', 'PETRA3 P09 EH2', 'PETRA3 P23 4C', 'PETRA3 P23 6C', 'SOLEIL MARS', 'SOLEIL NANOSCOPIUM ROBOT', 'SOLEIL SIRIUS KAPPA', 'SOLEIL SIRIUS TURRET', 'SOLEIL SIXS MED1+2', 'SOLEIL SIXS MED2+2', 'SOLEIL SIXS MED2+3', 'SOLEIL SIXS MED2+3 v2', 'TwoC', 'ZAXIS']


In [4]:
print(f"{fourc.solver.get()=}")
print(f"{fourc.geometry.get()=}")
print(f"{fourc.wavelength.get()=}")

fourc.solver.get()='hkl_soleil'
fourc.geometry.get()='E4CV'
fourc.wavelength.get()=1.0


In [5]:
print(f"{fourc.solver_name=}")
print(f"{fourc.operator.solver=}")
print(f"{fourc.operator.axes_xref=!r}")  # our names to solver's names
print(f"{fourc.pseudo_axis_names=}")  # our full ordered lists of names
print(f"{fourc.real_axis_names=}")
print(f"{fourc.operator.solver.pseudo_axis_names=}")  # solver's ordered lists of names
print(f"{fourc.operator.solver.real_axis_names=}")
print(f"{fourc.operator.solver.extra_axis_names=}")

fourc.solver_name='hkl_soleil'
fourc.operator.solver=HklSolver(name='hkl_soleil', version='v5.0.0.3434', geometry='E4CV', engine='hkl', mode='bissector')
fourc.operator.axes_xref={'h': 'h', 'k': 'k', 'l': 'l', 'theta': 'omega', 'chi': 'chi', 'phi': 'phi', 'ttheta': 'tth'}
fourc.pseudo_axis_names=['h', 'k', 'l', 'h2', 'k2', 'l2']
fourc.real_axis_names=['theta', 'chi', 'phi', 'ttheta', 'psi', 'energy']
fourc.operator.solver.pseudo_axis_names=['h', 'k', 'l']
fourc.operator.solver.real_axis_names=['omega', 'chi', 'phi', 'tth']
fourc.operator.solver.extra_axis_names=[]


In [6]:
print(f"{fourc.samples=}")
print(f"{fourc.sample=}")
fourc.operator.remove_sample("vibranium",)  # just in case it was defined previously
fourc.add_sample("vibranium", 2*math.pi, digits=3, replace=True)  # or force a replacement
print(f"{fourc.sample=}")
print(f"{fourc.samples=}")
fourc.sample = "cubic"  # switch back to the "cubic" sample
print(f"{fourc.sample=}")

fourc.samples={'cubic': Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))}
fourc.sample=Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))
fourc.sample=Sample(name='vibranium', lattice=Lattice(a=6.283, b=6.283, c=6.283, alpha=90.0, beta=90.0, gamma=90.0))
fourc.samples={'cubic': Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0)), 'vibranium': Sample(name='vibranium', lattice=Lattice(a=6.283, b=6.283, c=6.283, alpha=90.0, beta=90.0, gamma=90.0))}
fourc.sample=Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))


In [7]:
fourc.report  # FIXME: not yet implemented.  These are fake values.

{'position': FourcPseudoPos(h=0, k=0, l=0, h2=0, k2=0, l2=0)}

In [8]:
fourc.sample = "vibranium"
fourc.sample.reflections.ordering = []  # TODO: list of reflections could be made easier, still

In [9]:
fourc.sample.reflections.add(  # FIXME: not yet implemented.  Content ignored at this time.
    hklpy2.Reflection(
        fourc.operator.solver, 
        dict(h=1, k=0, l=0),   # TODO: be more flexible for input types here
        dict(omega=10, chi=0, phi=0, tth=20),  # TODO: be more flexible for input types here
        wavelength=1.00, 
        name="r1"
    ),
    replace=True
)
fourc.sample.reflections.add(  # FIXME: not yet implemented.  Content ignored at this time.
    hklpy2.Reflection(
        fourc.operator.solver, 
        dict(h=0, k=1, l=0), 
        dict(omega=10, chi=-90, phi=0, tth=20),  # TODO: such as get from current positioners
        wavelength=1.00,  # TODO: same here
        name="r2"
    ),
    replace=True
)
for r in fourc.sample.reflections.ordering:
    print(f"{fourc.sample.reflections[r]}")

Reflection(name='r1', geometry='E4CV', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r2', geometry='E4CV', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)


In [10]:
fourc.sample.reflections.swap()
print(f"{fourc.sample.reflections.ordering=}")
for r in fourc.sample.reflections.ordering:
    print(f"{fourc.sample.reflections[r]}")

fourc.sample.reflections.ordering=['r2', 'r1']
Reflection(name='r2', geometry='E4CV', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r1', geometry='E4CV', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)


In [11]:
fourc.summary()

data keys (* hints)
-------------------
*fourc_chi
*fourc_energy
*fourc_h
*fourc_h2
 fourc_h2_setpoint
 fourc_h_setpoint
*fourc_k
*fourc_k2
 fourc_k2_setpoint
 fourc_k_setpoint
*fourc_l
*fourc_l2
 fourc_l2_setpoint
 fourc_l_setpoint
*fourc_phi
*fourc_psi
*fourc_theta
*fourc_ttheta

read attrs
----------
h                    PseudoSingle        ('fourc_h')
h.readback           AttributeSignal     ('fourc_h')
h.setpoint           AttributeSignal     ('fourc_h_setpoint')
k                    PseudoSingle        ('fourc_k')
k.readback           AttributeSignal     ('fourc_k')
k.setpoint           AttributeSignal     ('fourc_k_setpoint')
l                    PseudoSingle        ('fourc_l')
l.readback           AttributeSignal     ('fourc_l')
l.setpoint           AttributeSignal     ('fourc_l_setpoint')
theta                SoftPositioner      ('fourc_theta')
chi                  SoftPositioner      ('fourc_chi')
phi                  SoftPositioner      ('fourc_phi')
ttheta               Sof

In [13]:
from hklpy2 import SimulatedE4CV

sim4c = SimulatedE4CV("", name="sim4c")
print(f"{sim4c.sample=}")
print(f"{sim4c.solver=}")
print(f"{sim4c.operator.axes_xref=}")

sim4c.sample=Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))
sim4c.solver=AttributeSignal(name='sim4c_solver', parent='sim4c', value=0.0, timestamp=1716505471.408135)
sim4c.operator.axes_xref={'h': 'h', 'k': 'k', 'l': 'l', 'omega': 'omega', 'chi': 'chi', 'phi': 'phi', 'tth': 'tth'}
